<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#새로-띄워진-창에서-로그인을-한다" data-toc-modified-id="새로-띄워진-창에서-로그인을-한다-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>새로 띄워진 창에서 로그인을 한다</a></span><ul class="toc-item"><li><span><a href="#단어시험-함수" data-toc-modified-id="단어시험-함수-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>단어시험 함수</a></span></li><li><span><a href="#복습테스트-함수" data-toc-modified-id="복습테스트-함수-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>복습테스트 함수</a></span></li><li><span><a href="#pds-함수" data-toc-modified-id="pds-함수-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>pds 함수</a></span></li></ul></li></ul></div>

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display
import time
import requests
import re
import glob
import os



driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(10)
driver.get('https://nid.naver.com/oauth2.0/authorize?svctype=0&response_type=code&client_id=C9hwybENgOtF&state=IWK3A3NUJQVTFVU2KJPK2DRAZ6L7OXGBFYZHHTH3GMTTXKI6A6ALVCF6SFNE3OLHQ4JVWLE6HFC7Q===&redirect_url=https%3A%2F%2Fauth.band.us%2Fexternal_account_login%3Ftype%3Dnaver')


# 새로 띄워진 창에서 로그인을 한다

## 단어시험 함수

In [39]:
def voca(nums):
    
    field = '단어시험'
    
    for class_id in list(class_dic.keys()):
        
        student_data = []
        
        for num in nums:
           
            tagnum = f'{num}회'
            tag = f'{field}{tagnum}'

            url = class_dic[class_id] + f'/hashtag/{tag}'
            driver.get(url)
            time.sleep(1)

            try: 
                driver.find_element_by_class_name('commentToggle._commentToggleBtn').click()
                time.sleep(1)

                for _ in range(3):
                    try:
                        driver.find_element_by_class_name('prevComment').click()
                        time.sleep(0.2)
                    except :
                        pass



                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                students = soup.select('.itemWrap')


                for student in students :
                    name = student.select('button > strong')[0].text
                    if class_id == 'PURPLE':    # 퍼플은 반명을 한글로 적는 바람에 따로 해줘야 함 
                        name = name[5:].strip() # 전부 다 이거로 하게 되면 띄어쓰기 없는 애들은 솔팅 안됨 
                    try :
                        score = student.select('p')[0].text #이거 되는지 마지막으로 확인할 것 
                    except :
                        score = ""

                    student_data.append([class_id, field, tagnum, name, score])

                print(len(student_data))
            
            except:
                pass

        columns = ['반명', '숙제이름', '회차', '이름', '점수']
        student_list = pd.DataFrame(student_data, columns=columns)
        student_list = student_list.drop_duplicates()
        student_list['이름'] = student_list['이름'].str.extract(r'([가-힣].*)') # 한글로 시작하는 시점부터 뒤까지 전부 불러와라 
        student_list.to_excel(f'{class_id}_{field}.xlsx', index=False)


In [37]:
class_dic = {}
# class_dic['PA3A'] = 'https://band.us/band/84576914'
# class_dic['PB3A'] = 'https://band.us/band/84576918'
# class_dic['PD3A'] = 'https://band.us/band/84576926'
# class_dic['PJ3A'] = 'https://band.us/band/84576928'
# class_dic['PURPLE'] = 'https://band.us/band/84570800'
# class_dic['RED'] = 'https://band.us/band/84576846'
# class_dic['PB7A'] = 'https://band.us/band/84600978'
# class_dic['PD7A'] = 'https://band.us/band/84601004'

In [35]:
# 단어시험 
nums = list(range(1, 14))
voca(field, nums)

55
109
161
211
260
309
350
390
434
477
521
557
597


## 복습테스트 함수

In [ ]:
def re_test(tag):
    
    for class_id in list(class_dic.keys()):
        url = class_dic[class_id] + f'/hashtag/{tag}'
        driver.get(url)


        # 밴드 이름 가져오기 
        band_name = driver.find_element_by_class_name('uriText').text


        # 퀴즈박스 클릭
        driver.find_element_by_class_name('item.-quiz').click() 


        # 퀴즈명 가져오기 
        quiz_name = driver.find_element_by_xpath('//*[@id="wrap"]/div[3]/div/div/section/div[2]/div/section/div/div[3]/div[3]/div[4]/div/div/p[2]')
        quiz_name = quiz_name.text.replace("~", "_")


        scroll = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '//*[@id="wrap"]/div[3]/div/div/section')))
        driver.execute_script("arguments[0].scrollBy(0,arguments[0].scrollHeight)", scroll) # t스크롤

        driver.find_element_by_class_name('button._btnOpenTakerList').click() # 응답자 보기 클릭
        driver.find_element_by_class_name('btnMoreMenu._btnHeaderMoreMenu').click() # 점 3개 클릭
        driver.find_element_by_class_name('btnMenuText._btnDownloadGradingResult').click() # 채점결과 다운로드 

        time.sleep(2)


        # 저장된 파일 불러오기 
        add = f'C:/Users/hmkang/Downloads/{band_name}_채점결과_{quiz_name}_' + r'{}_{}.xlsx'
        filename = glob.glob(add.format("*", "*"))[0]
        rere = pd.read_excel(filename, header=3)

        # 멤버 및 점수만 뽑아내기 
        rere['멤버'] = rere['멤버'].str.extract(r'([가-힣].*)')
        rere = rere.dropna()
        rere = rere[['멤버', '점수']]
        return rere

In [ ]:
tag = '온택트복습TEST'
re_test(tag)

## pds 함수

In [40]:
def pds(nums):
    
    field = 'PDS복습'
    
    for class_id in list(class_dic.keys()):
        
        student_data = []
        
        for num in nums:
            
            tagnum = f'{num}차'

            tag = f'{field}{tagnum}'

            url = class_dic[class_id] + f'/hashtag/{tag}'
            driver.get(url)
            time.sleep(1)

            try: 
                driver.find_element_by_class_name('commentToggle._commentToggleBtn').click()
                time.sleep(2)

                for _ in range(4):
                    try:
                        driver.find_element_by_class_name('prevComment').click()
                        time.sleep(0.2)
                    except :
                        pass



                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                students = soup.select('.itemWrap')


                for student in students :

                    name = student.select('button > strong')[0].text
                    if class_id == 'PURPLE':    # 퍼플은 반명을 한글로 적는 바람에 따로 해줘야 함 
                        name = name[5:].strip() # 전부 다 이거로 하게 되면 띄어쓰기 없는 애들은 솔팅 안됨 
                    score = 'O'

                    student_data.append([class_id, field, tagnum, name, score])

                print(len(student_data))
            
            except:
                pass

        columns = ['반명', '숙제이름', '회차', '이름', '점수']
        student_list = pd.DataFrame(student_data, columns=columns)
        student_list = student_list.drop_duplicates()
        student_list['이름'] = student_list['이름'].str.extract(r'([가-힣].*)') # 한글로 시작하는 시점부터 뒤까지 전부 불러와라 
        student_list.to_excel(f'{class_id}_{field}.xlsx', index=False)

In [ ]:
nums = list(range(1, 14))
pds(nums)

In [ ]:
students

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
students = soup.select('.itemWrap')


In [ ]:
data = []
for student in students :
    name = student.select('button > strong')[0].text
    if class_id == 'PURPLE':
        name = name[5:].strip()
    try :
        score = student.select('div.commentBody._commentModification > p')[0].text
    except :
        score =""
    data.append([name, score])
data

In [ ]:
students

In [ ]:
name = student.select('button > strong')[0].text
name

In [ ]:
score = student.select('p')[0].text

In [ ]:
students

In [ ]:
a = driver.find_element_by_css_selector('div.itemWrap > div.writeInfo > button > strong')

In [ ]:
student

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


In [ ]:
pds = soup.select('div.itemWrap > div.writeInfo > button')

In [ ]:
name_list = []
# pds_n = input('태그를 입력하세요 :')
pds_n = 'PDS복습34차'

for name in pds: 
    who = name.text
    who = who.replace('\n', "")
    where, who = who.split(" ")
    what = pds_n

    name_list.append([what, where, who])

    
df = pd.DataFrame(name_list, columns=['숙제이름', '반명', '이름'])
df

In [ ]:
class_dic

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')




In [ ]:
# class_number = input("반명은? : ")
url = class_dic[class_number]+'/member'
# driver.get(url)
a = list(class_dic.keys())
a

In [ ]:
class_dic = {}
class_dic['PA3A'] = 'https://band.us/band/84576914'
class_dic['PB3A'] = 'https://band.us/band/84576918'
class_dic['PD3A'] = 'https://band.us/band/84576926'
class_dic['PJ3A'] = 'https://band.us/band/84576928'
class_dic['PURPLE'] = 'https://band.us/band/84570800'
class_dic['RED'] = 'https://band.us/band/84576846'
class_dic['PB7A'] = 'https://band.us/band/84600978'
class_dic['PD7A'] = 'https://band.us/band/84601004'


# class_number = input("반명은? : ")
member_list = []
for class_number in list(class_dic.keys()):
    url = class_dic[class_number]+'/member'
    driver.get(url)
    time.sleep(2)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    member = soup.select('span.text.-flex')

    for name in member[1:]:
        who = name.text
        who = who.replace('\n', "")
        if class_number == 'PURPLE':
            who = who[5:].strip()
        where = class_number
        member_list.append([where, who])

member11 = pd.DataFrame(member_list, columns=['반명', '이름'])
member11['이름']=member11['이름'].str.extract(r'([가-힣].*)')
member11.to_excel('student_08.xlsx', index=False)

In [ ]:
member11['이름']=member11['이름'].str.extract(r'([가-힣].*)')

member11

In [ ]:
member11.to_excel('student_08.xlsx', index=False)

In [ ]:
class_dic = {}
class_dic['PA3A'] = 'https://band.us/band/84576914'
class_dic['PB3A'] = 'https://band.us/band/84576918'
class_dic['PD3A'] = 'https://band.us/band/84576926'
class_dic['PJ3A'] = 'https://band.us/band/84576928'
class_dic['PURPLE'] = 'https://band.us/band/84570800'
class_dic['RED'] = 'https://band.us/band/84576846'
class_dic['PB7A'] = 'https://band.us/band/84600978'
class_dic['PD7A'] = 'https://band.us/band/84601004'


In [ ]:
driver.add

In [ ]:
s = requests.Session()

In [ ]:
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'
}
s.headers.update(headers)

In [ ]:
for cookie in driver.get_cookies():
    c = {cookie['name'] : cookie['value']}
    s.cookies.update(c)

In [ ]:
s.cookies


In [ ]:
response = s.get('https://band.us/band/84576914/hashtag/PDS%EB%B3%B5%EC%8A%B532%EC%B0%A8')
soup = BeautifulSoup(response.text, 'lxml')
soup

In [ ]:
for cookie in driver.get_cookies():
    driver.add_cookie(c)

In [ ]:
cks = driver.get_cookies()
cks